# 檢查資料 ubike-utf8.csv 是否已在

In [7]:
#!/bin/env python
# -*- coding: utf-8 -*-

import csv
from datetime import datetime
import psycopg2

def dateime_strptime_or_none(x, format_str):
    try:
        return datetime.strptime(x, format_str)
    except ValueError:
        return None

def str_or_none(x):
    if x == 'NA':
        return None
    return x

def float_or_none(x):
    if x == 'NA':
        return None
    return float(x)

def load_ubike():

    conn = psycopg2.connect(user='lab')
    cur = conn.cursor()

    cur.execute('''drop table if exists ubike''')

    cur.execute('''
        create table ubike (
            when_ts       timestamp,
            where_pt      point,
            code          varchar(8),
            name          varchar(32),
            area_name     varchar(32),
            space_num     int,
            avg_bike_num  real,
            max_bike_num  int,
            min_bike_num  int,
            bike_num_std  real,
            avg_space_num real,
            max_space_num int,
            min_space_num int,
            space_num_std real
        )
    ''')
    cur.execute('''create index on ubike (when_ts)''')
    cur.execute('''create index on ubike (code)''')
    cur.execute('''create index on ubike (name)''')
    cur.execute('''create index on ubike using gist (where_pt)''')

    ubike_csv_reader = csv.reader(open('ubike-utf8.csv'))
    # skip title
    next(ubike_csv_reader)
    for row in ubike_csv_reader:
        cur.execute('''insert into ubike values (
            %s,
            '%s',
            %s, %s, %s, %s,
            %s, %s, %s, %s,
            %s, %s, %s, %s
        )''', (
            dateime_strptime_or_none(row[0]+' '+row[1], '%Y-%m-%d %H'),
            # where_pt
            (float(row[5]), float(row[6])),
            # code, name, area_name, space_num
            row[2], str_or_none(row[4]), row[3], int(row[7]),
            # avg_bike_num~
            float(row[8]), int(row[9]), int(row[10]), float_or_none(row[11]),
            # avg_space_num~
            float(row[12]), int(row[13]), int(row[14]), float_or_none(row[15])
        ))

    cur.close()
    conn.commit()
    conn.close()

if __name__ == '__main__':
    load_ubike()


# 重新連線 上面的 conn.close() 會中斷連線釋放資源

In [16]:
    conn = psycopg2.connect(user='lab')
    cur = conn.cursor()

In [18]:
 bike = cur.execute("select 'code' from ubike limit 3;")

# 使用 pandas 的相關sql 語法庫

In [1]:
import pandas.io.sql

In [28]:
# query db
sql = """

select * from ubike limit 3

"""
df = pandas.io.sql.read_sql(sql, conn)
df.head()

,when_ts,where_pt,code,name,area_name,space_num,avg_bike_num,max_bike_num,min_bike_num,bike_num_std,avg_space_num,max_space_num,min_space_num,space_num_std
0,2014-12-08 15:00:00,"(25.041,121.556945)",2,捷運國父紀念館站(2號出口),大安區,48,24.000,27,23,1.549,22.000,23,19,2
1,2014-12-08 15:00:00,"(25.037797,121.565169)",3,台北市政府,信義區,40,10.333,13,8,1.862,29.667,32,27,2
2,2014-12-08 15:00:00,"(25.036036,121.562325)",4,市民廣場,信義區,60,39.333,40,38,1.033,20.667,22,20,1


# 檢查資料

In [29]:
df.columns

Index([u'when_ts', u'where_pt', u'code', u'name', u'area_name', u'space_num', u'avg_bike_num', u'max_bike_num', u'min_bike_num', u'bike_num_std', u'avg_space_num', u'max_space_num', u'min_space_num', u'space_num_std'], dtype='object')

In [30]:
df.dtypes

when_ts          datetime64[ns]
where_pt                 object
code                     object
name                     object
area_name                object
space_num                 int64
avg_bike_num            float64
max_bike_num              int64
min_bike_num              int64
bike_num_std            float64
avg_space_num           float64
max_space_num             int64
min_space_num             int64
space_num_std           float64
dtype: object

# 檢查資料 weathertp-big5-utf8.csv 是否已在

In [1]:
#!/bin/env python
# -*- coding: utf-8 -*-

import csv
from datetime import datetime
import psycopg2

def dateime_strptime_or_none(x, format_str):
    try:
        return datetime.strptime(x, format_str)
    except ValueError:
        return None

def str_or_none(x):
    if x == 'NA':
        return None
    return x

def float_or_none(x):
    if x == 'NA':
        return None
    return float(x)

def load_tpweather():

    conn = psycopg2.connect(user='lab')
    cur = conn.cursor()

    cur.execute('''drop table if exists tpweather''')

    cur.execute('''
        create table tpweather (
            when_ts    timestamp,
            where_pt   point,
            name       varchar(32),
            temp       real,
            max_temp   real,
            min_temp   real,
            hum_pct    real,
            pressure   real,
            wind_speed real,
            rainfall   real
        )
    ''')
    cur.execute('''create index on tpweather (when_ts)''')
    cur.execute('''create index on tpweather (name)''')
    cur.execute('''create index on tpweather using gist (where_pt)''')

    tpweather_csv_reader = csv.reader(open('weathertp-big5-utf8.csv'))
    # skip title
    next(tpweather_csv_reader)
    for row in tpweather_csv_reader:
        cur.execute('''insert into tpweather values (
            %s,
            '%s',
            %s,
            %s, %s, %s,
            %s, %s, %s, %s
        )''', (
            dateime_strptime_or_none(row[0]+' '+row[1], '%Y-%m-%d %H:%M'),
            # where_pt
            (float(row[10]), float(row[11])),
            # name
            row[2],
            # temp~
            float_or_none(row[3]), float_or_none(row[4]), float_or_none(row[5]),
            # hum_pct~
            float_or_none(row[6]), float_or_none(row[7]), float_or_none(row[8]), float_or_none(row[9])
        ))

    cur.close()
    conn.commit()
    conn.close()

if __name__ == '__main__':
    load_tpweather()


# 再次連線並且檢查資料

In [3]:
import pandas.io.sql
import psycopg2
conn = psycopg2.connect(user='lab')
cur = conn.cursor()

In [4]:
# query db
sql = """

select * from tpweather limit 3

"""

tpweather_df = pandas.io.sql.read_sql(sql, conn)
tpweather_df.head()

,when_ts,where_pt,name,temp,max_temp,min_temp,hum_pct,pressure,wind_speed,rainfall
0,2014-12-19,"(25.0631782,121.5642857)",三民國小,15.6667,15.7,15.6,81,1023.160,2.2,0
1,2014-12-19,"(25.1180133,121.5373439)",三玉國小,17.3889,17.4,16.9,76,1022.380,2.7,0
2,2014-12-19,"(25.154895,121.503983)",大屯國小,14.7222,14.7,14.1,81,992.144,1.3,0


In [5]:
tpweather_df.columns

Index([u'when_ts', u'where_pt', u'name', u'temp', u'max_temp', u'min_temp', u'hum_pct', u'pressure', u'wind_speed', u'rainfall'], dtype='object')

In [6]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import psycopg2

conn = psycopg2.connect(user='lab')
cur = conn.cursor()

cur.execute('''drop table if exists ubike_nearest_tpweather''')

cur.execute('''
    create table ubike_nearest_tpweather (
        u_code varchar(8),
        w_name varchar(32),
        primary key (u_code, w_name)
    )
''')

cur.execute('''
    insert into
        ubike_nearest_tpweather
    select distinct on (u.code)
        code,
        (
            select name
            from tpweather as w
            order by w.where_pt<->u.where_pt
            limit 1
        )
    from ubike as u
''')

cur.close()
conn.commit()
conn.close()


In [10]:
conn = psycopg2.connect(user='lab')
cur = conn.cursor()

# query db
sql = """

select * from ubike_nearest_tpweather limit 3

"""

tpweather_df = pandas.io.sql.read_sql(sql, conn)
tpweather_df.head()

,u_code,w_name
0,1,博愛國小
1,10,吳興國小
2,100,忠孝國小
